In [1]:
import numpy as np
import ctypes
import subprocess

# First, compiple EIQP_p.c to EIQP.so in Linux
cmd = ["gcc", "EIQP_p.c", "-o", "EIQP.so", "-O3", "-lblas", "-llapack", "-llapacke", "-shared", "-fPIC"]
subprocess.run(cmd)

# Second, define the Python structure related to the output of EIQP solver
clib = ctypes.CDLL("./EIQP.so")
clib.EIQP.argtypes = [
    ctypes.POINTER(ctypes.c_double), # input: Q
    ctypes.POINTER(ctypes.c_double), # input: c
    ctypes.POINTER(ctypes.c_double), # input: A
    ctypes.POINTER(ctypes.c_double), # input: b
    ctypes.c_double, # input: epsilon
    ctypes.c_int, # n
    ctypes.c_int, # m
    ctypes.POINTER(ctypes.c_double) # output: z
]
clib.EIQP.restype = ctypes.c_int # output: status

In [2]:
n = 6
m = 6
Q = np.eye(n)
c = np.array([[-1.0],[-0.8],[-0.6],[-0.4],[-0.2],[0.0]])
A = -np.eye(m)
b = -np.ones(m)
z = np.zeros(Q.shape[0],dtype=np.float64)

# Third, here is to call EIQP
Q_ptr = Q.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
c_ptr = c.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
A_ptr = A.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
b_ptr = b.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
z_ptr = z.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
epsilon = 1e-8
status = clib.EIQP(Q_ptr,c_ptr,A_ptr,b_ptr,epsilon,n,m,z_ptr)

# Last, show the result
print(status)
print(z)


1
[9.99980090e-01 7.99999998e-01 5.99999999e-01 4.00000000e-01
 2.00000001e-01 1.99100987e-05]
